SELECT
  ot.order_id,
  dm.dealer_id,
  di.material_no,
  di.date,
	CASE WHEN ot.order_date = '0000-00-00' THEN di.date ELSE ot.order_date END AS ORDER_DATE,
  di.state_id,
  SUM(di.quantity) AS total_quantity,
  opt.rate AS FINAL_PRICE
FROM dispatch_item di
INNER JOIN order_tbl ot ON di.reference = ot.referenceno
LEFT JOIN order_prod_tbl opt ON ot.order_id = opt.order_id AND di.material_no = opt.material_code
LEFT JOIN dealer_master dm ON di.sap_id = dm.sapid
INNER JOIN product_dealer_tbl pdt ON di.material_no = pdt.productCode
WHERE ot.d_status = 0
  AND dm.d_status = 0
  AND pdt.in_app_show = 1
  AND dm.dealer_type NOT IN ('Survey','Rejected','Dormant')
  AND opt.rate > 0 AND opt.rate IS NOT NULL
  AND pdt.d_status = 0
  AND DATE(di.date) < '2024-12-01' AND DATE(di.date) > '2023-03-31'
GROUP BY
  di.material_no,
  ORDER_DATE,
  di.date,
  di.state_id,
  opt.rate,    
  ot.order_id  
ORDER BY di.date ASC

**Import Neccessary Libraries**

In [1]:
import warnings
warnings.filterwarnings("ignore")

#for data manipulation
import pandas as pd
import numpy as np
from itertools import product

#for data visualisation
import matplotlib.pyplot as plt
import seaborn as sns

#for data modelling
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
# from sklearn import svm
import xgboost as xgb
import pickle

#for model evaluation
from sklearn.metrics import silhouette_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score

In [2]:
# Read the festival data
festival_df = pd.read_csv('festival_data.csv')

# Read the dealer data
dealer_df = pd.read_csv('UBA_F.csv')

# Read off season data
season_df = pd.read_csv('off_season_data.csv')

**Introductory Insights**

In [3]:
dealer_df.head()

,order_id,dealer_id,material_no,date,ORDER_DATE,state_id,total_quantity,FINAL_PRICE
0,40942,2188104,4000071,2023-04-01,2023-04-01,7,6.02,80600.0
1,40941,3972,4000071,2023-04-01,2023-04-01,7,3.02,80600.0
2,40900,28078,4000071,2023-04-01,2023-04-01,7,5.05,80600.0
3,40899,2147568,4000071,2023-04-01,2023-03-30,4,11.01,75300.0
4,40865,2267173,4000071,2023-04-01,2023-04-01,5,11.02,79000.0


In [4]:
festival_df.head()

,Date,FestivalName,TypeofFestival
0,26-01-2019,RepublicDay,National
1,10-02-2019,SaraswatiPuja,National
2,04-03-2019,MahaShivaratri,State/National
3,21-03-2019,Holi,National
4,06-04-2019,Ugadi,State


In [5]:
season_df.head()

,weather_date,weather_name,off_season
0,01-04-2022,Spring,0
1,02-04-2022,Spring,0
2,03-04-2022,Spring,0
3,04-04-2022,Spring,0
4,05-04-2022,Spring,0


In [6]:
dealer_df.shape

(102043, 8)

In [7]:
festival_df.shape

(206, 3)

In [8]:
season_df.shape

(2192, 3)

**Data Preprocessing**

In [9]:
# Convert the 'Date' column in festival data and 'ORDER_DATE' column in dealer data to datetime format
festival_df['Date'] = pd.to_datetime(festival_df['Date'], errors='coerce')
dealer_df['ORDER_DATE'] = pd.to_datetime(dealer_df['ORDER_DATE'], errors='coerce')

# Create a set of festival dates for fast lookup
festival_dates = set(festival_df['Date'])

# Function to check if the ORDER_DATE is a festival date
def check_festival(order_date):
    return 1 if order_date in festival_dates else 0

# Apply the function to the 'ORDER_DATE' column to create the 'festival_flag' column
dealer_df['festival_flag'] = dealer_df['ORDER_DATE'].apply(check_festival)

In [10]:
dealer_df

,order_id,dealer_id,material_no,date,ORDER_DATE,state_id,total_quantity,FINAL_PRICE,festival_flag
0,40942,2188104,4000071,2023-04-01,2023-04-01,7,6.020,80600.0,0
1,40941,3972,4000071,2023-04-01,2023-04-01,7,3.020,80600.0,0
2,40900,28078,4000071,2023-04-01,2023-04-01,7,5.050,80600.0,0
3,40899,2147568,4000071,2023-04-01,2023-03-30,4,11.010,75300.0,1
4,40865,2267173,4000071,2023-04-01,2023-04-01,5,11.020,79000.0,0
...,...,...,...,...,...,...,...,...,...
102038,74647,22225,4400001,2024-11-30,2024-11-30,5,0.530,75100.0,0
102039,74635,2188240,4400001,2024-11-30,2024-11-30,5,0.065,75100.0,0
102040,74578,232927688,4400001,2024-11-30,2024-11-30,5,0.500,75100.0,0
102041,74575,2207288,4400001,2024-11-30,2024-11-30,19,0.194,79900.0,0


In [11]:
# Convert the 'weather_date' column in festival data and 'ORDER_DATE' column in dealer data to datetime format
season_df['weather_date'] = pd.to_datetime(season_df['weather_date'], errors='coerce')
dealer_df['ORDER_DATE'] = pd.to_datetime(dealer_df['ORDER_DATE'], errors='coerce')

# Create a set of festival dates for fast lookup
season_dates = set(season_df['weather_date'])

# Function to check if the ORDER_DATE is a festival date
def check_season(order_date):
    if order_date in season_dates:
        return season_df.loc[season_df['weather_date'] == order_date, 'off_season'].values[0]
    else:
        return 0  # or any default value indicating not off-season

# Apply the function to the 'ORDER_DATE' column to create the 'season_flag' column
dealer_df['season_flag'] = dealer_df['ORDER_DATE'].apply(check_season)

# Output the result
dealer_df.to_csv('dealer_data_with_season_flag.csv', index=False)

print("Season flag added and saved to 'dealer_data_with_season_flag.csv'")

Season flag added and saved to 'dealer_data_with_season_flag.csv'


In [12]:
# Convert the 'request_date' column to datetime, with day first
dealer_df['date'] = pd.to_datetime(dealer_df['ORDER_DATE'], dayfirst=True)

# Extract the year and month into separate columns
dealer_df['year'] = dealer_df['date'].dt.year
dealer_df['month'] = dealer_df['date'].dt.month

# Display the DataFrame to confirm the changes
dealer_df

,order_id,dealer_id,material_no,date,ORDER_DATE,state_id,total_quantity,FINAL_PRICE,festival_flag,season_flag,year,month
0,40942,2188104,4000071,2023-04-01,2023-04-01,7,6.020,80600.0,0,0,2023,4
1,40941,3972,4000071,2023-04-01,2023-04-01,7,3.020,80600.0,0,0,2023,4
2,40900,28078,4000071,2023-04-01,2023-04-01,7,5.050,80600.0,0,0,2023,4
3,40899,2147568,4000071,2023-03-30,2023-03-30,4,11.010,75300.0,1,0,2023,3
4,40865,2267173,4000071,2023-04-01,2023-04-01,5,11.020,79000.0,0,0,2023,4
...,...,...,...,...,...,...,...,...,...,...,...,...
102038,74647,22225,4400001,2024-11-30,2024-11-30,5,0.530,75100.0,0,0,2024,11
102039,74635,2188240,4400001,2024-11-30,2024-11-30,5,0.065,75100.0,0,0,2024,11
102040,74578,232927688,4400001,2024-11-30,2024-11-30,5,0.500,75100.0,0,0,2024,11
102041,74575,2207288,4400001,2024-11-30,2024-11-30,19,0.194,79900.0,0,0,2024,11


In [13]:
dealer_df.sort_values(['ORDER_DATE'])

,order_id,dealer_id,material_no,date,ORDER_DATE,state_id,total_quantity,FINAL_PRICE,festival_flag,season_flag,year,month
713,39622,2012845,4000071,2023-03-13,2023-03-13,3,4.040,78400.0,0,0,2023,3
720,39622,2012845,4000072,2023-03-13,2023-03-13,3,3.030,77400.0,0,0,2023,3
725,39622,2012845,4000073,2023-03-13,2023-03-13,3,1.060,76400.0,0,0,2023,3
1321,39622,2012845,4000074,2023-03-13,2023-03-13,3,2.019,76400.0,0,0,2023,3
132,40213,2157220,4200010,2023-03-27,2023-03-27,4,1.000,85300.0,1,0,2023,3
...,...,...,...,...,...,...,...,...,...,...,...,...
101420,74620,2049851,4000072,2024-11-30,2024-11-30,7,2.150,73500.0,0,0,2024,11
101421,74619,50529,4000072,2024-11-30,2024-11-30,5,2.000,71100.0,0,0,2024,11
101422,74613,2176690,4000072,2024-11-30,2024-11-30,19,1.006,75900.0,0,0,2024,11
101424,74600,2258354,4000072,2024-11-30,2024-11-30,4,2.980,67000.0,0,0,2024,11


In [14]:
# df = dealer_df.drop(columns=['reference', 'quantity', 'stock_id','qnty_in_kg', 'date', 'material_no', 'nod', 'state_id', 'dealer_id', 'PRICE_TYPE'])
df = dealer_df.drop(columns=['total_quantity', 'date','order_id','ORDER_DATE'])

In [15]:
def categorize_price(x):
    if 40000 <= x <= 50000:
        return 1
    elif 50001 <= x <= 60000:
        return 2
    elif 60001 <= x <= 70000:
        return 3
    elif 70001 <= x <= 80000:
        return 4
    elif 80001 <= x <= 90000:
        return 5
    elif 90001 <= x <= 100000:
        return 6
    elif x > 100001:
        return 7
    else:
        return 8

df['range'] = df['FINAL_PRICE'].apply(categorize_price)


In [16]:
# df = df.reset_index()

In [17]:
df = df.drop(columns=['FINAL_PRICE'])

In [ ]:
df

,dealer_id,material_no,state_id,festival_flag,season_flag,year,month,range
0,2188104,4000071,7,0,0,2023,4,5
1,3972,4000071,7,0,0,2023,4,5
2,28078,4000071,7,0,0,2023,4,5
3,2147568,4000071,4,1,0,2023,3,4
4,2267173,4000071,5,0,0,2023,4,4
...,...,...,...,...,...,...,...,...
102038,22225,4400001,5,0,0,2024,11,4
102039,2188240,4400001,5,0,0,2024,11,4
102040,232927688,4400001,5,0,0,2024,11,4
102041,2207288,4400001,19,0,0,2024,11,4


In [19]:
X = df.drop(columns=['dealer_id'])
Y = df['dealer_id']

In [20]:
# # Step 3: Train-Test Split
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [21]:
 # First, split the data into training and temp (which will be further split into validation and test)
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.4, random_state=42)
 
# Now split the temp data into validation and test sets
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)
 
# Output the sizes of the splits
print(f"Training set size: {X_train.shape[0]}")
print(f"Validation set size: {X_val.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

Training set size: 61225
Validation set size: 20409
Test set size: 20409


In [22]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape, X_temp.shape, Y_temp.shape, X_val.shape, Y_val.shape

((61225, 7),
 (20409, 7),
 (61225,),
 (20409,),
 (40818, 7),
 (40818,),
 (20409, 7),
 (20409,))

**Random Forest Classifier**

In [23]:
# rfc_model = RandomForestClassifier()
# rfc_model.fit(X_train, Y_train)
# Y_pred_rfc = rfc_model.predict(X_test)

**Decision Tree Classifier**

In [24]:
# dtc_model = DecisionTreeClassifier()
# dtc_model.fit(X_train, Y_train)
# Y_pred_dtc = dtc_model.predict(X_test)

In [25]:
# # Select a subset of dealer_ids to reduce memory usage and still get a big plot
# top_dealers = df['dealer_id'].value_counts().nlargest(10).index  # Top 50 dealers by frequency

# # Modify Y_test and Y_pred to include only the top dealers
# Y_test_sub = [y if y in top_dealers else -1 for y in Y_test]  # Modify Y_test to include only top dealers
# Y_pred_sub = [y if y in top_dealers else -1 for y in Y_pred_dtc]

# # Create confusion matrix with reduced labels
# cm = confusion_matrix(Y_test_sub, Y_pred_sub, labels=top_dealers)

# # Create the display
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=top_dealers)

# # Set up a large figure (e.g., 15x15 inches)
# fig, ax = plt.subplots(figsize=(20, 20)) 

# # Plot the confusion matrix
# disp.plot(cmap=plt.cm.Blues, ax=ax)

# # Customize the plot
# plt.title('Confusion Matrix', fontsize=20, pad=30)
# plt.xlabel('Prediction', fontsize=15)
# plt.ylabel('Actual', fontsize=15)

# # Further customizations
# plt.gca().xaxis.set_label_position('top')
# plt.gca().xaxis.tick_top()
# plt.gca().figure.subplots_adjust(bottom=0.2)

# plt.show()


In [26]:
# dtc_acc = accuracy_score(Y_test, Y_pred_dtc)
# dtc_acc

In [27]:
# rfc_acc = accuracy_score(Y_test, Y_pred_rfc)
# rfc_acc

In [28]:
df.head()

,dealer_id,material_no,state_id,festival_flag,season_flag,year,month,range
0,2188104,4000071,7,0,0,2023,4,5
1,3972,4000071,7,0,0,2023,4,5
2,28078,4000071,7,0,0,2023,4,5
3,2147568,4000071,4,1,0,2023,3,4
4,2267173,4000071,5,0,0,2023,4,4


In [29]:
X.head()

,material_no,state_id,festival_flag,season_flag,year,month,range
0,4000071,7,0,0,2023,4,5
1,4000071,7,0,0,2023,4,5
2,4000071,7,0,0,2023,4,5
3,4000071,4,1,0,2023,3,4
4,4000071,5,0,0,2023,4,4


In [30]:
Y.head()

0    2188104
1       3972
2      28078
3    2147568
4    2267173
Name: dealer_id, dtype: int64

KMeans

In [ ]:
dealervalues = df['dealer_id'].unique()  # Get unique dealer IDs
n_clusters = len(dealervalues)  # Number of clusters will be the number of unique dealer IDs
 
# Step 1: Fit the KMeans model with number of clusters equal to the number of dealer values
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df['cluster'] = kmeans.fit_predict(X)  # Assign clusters directly to the df
 
df['order_pred'] = df['cluster']  # Assuming each dealer gets their own cluster
 
# Step 3: Inertia (sum of squared distances to the cluster centers)
print(f"Inertia: {kmeans.inertia_}")  # Lower is better
 
# Step 4: Silhouette Score
sil_score = silhouette_score(X, kmeans.labels_)  # Evaluates how well-defined the clusters are
print(f"Model Score: {sil_score:.4f}")  # Closer to 1 is better, and a negative score is bad

Inertia: 4109.810828251051


In [43]:
# Function to predict whether dealers will buy or not based on input parameters
def predict_dealer_order(dealer_id, material_no, state_id, festival_flag, season_flag, year, month, range):
    # Create a DataFrame for the input parameters
    input_data = pd.DataFrame({
        # 'dealer_id': [dealer_id],
        'material_no': [material_no],
        'state_id': [state_id],
        'festival_flag': [festival_flag],
        'season_flag': [season_flag],
        'year': [year],
        'month': [month],
        'range': [rangeval]
    })
    
    # Ensure input_data has the same columns as X used during training
    X_input = input_data[['material_no', 'state_id', 'festival_flag', 'season_flag', 'year', 'month', 'range']]
    
    # Predict using the trained KMeans model
    order_pred = kmeans.predict(X_input)
    
    return order_pred

# Example usage
# dealer_id = 2188104
state_id = 7
material_no = 4000071
year = 2024
month = 12
rangeval = 5
season_flag = 0
festival_flag = 0

prediction = predict_dealer_order(dealer_id, material_no, state_id, festival_flag, season_flag, year, month, range)
print(f"Predicted Dealer: {dealer_id} from state {state_id} will buy this {material_no} on {month}-{year}")

Predicted Dealer: 2188104 from state 7 will buy this 4000071 on 12-2024
